In [1]:
# Import required packages
import torch

from utils.solver import Solver, DEFAULT_SOLVER_PARAM_M7
from utils.robot import get_robot

In [2]:
num_poses = 100
num_sols = 100

In [3]:
robot = get_robot()
solver = Solver(robot=robot, solver_param=DEFAULT_SOLVER_PARAM_M7)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [4]:
# solver.latent = torch.zeros(1, 7).cuda()
# solver.shirnk_ratio = 0
solver.random_evaluation(num_poses, num_sols, return_time=True)

(0.01012909301506167, 0.10595002555847167, 0.29)

In [5]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

config.py: Using device 'cuda:0'


In [6]:
set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

P = solver._random_sample_poses(num_poses)

set_seed() - random int:  541
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tr

In [7]:
# -> unrefined solutions
l2_errs = np.zeros((len(P), num_sols))
ang_errs = np.zeros((len(P), num_sols))

time_diffs = np.zeros((len(P)))

for i, p in enumerate(P):
    solutions, l2_errors, angular_errors, joint_limits_exceeded, self_colliding, runtime = ik_solver.solve(
        p, 
        n=num_sols,
        refine_solutions=False, 
        return_detailed=True)

    solutions = solutions.detach().cpu().numpy()
    
    l2_errs[i] = l2_errors
    ang_errs[i] = angular_errors
    time_diffs[i] = runtime

l2_errs = l2_errs.flatten()

l2_errs.mean(), time_diffs.mean()

(0.008127666906669651, 0.07917458772659301)